# improts

In [1]:
# imports
import requests
import pandas as pd
from notion_client import Client
import datetime
import json
import time

Setup

In [2]:
# BLENDAPI setup
BLEND_API_KEY = "org_ba2e4ccfb75e56afc088d9804df57d2623542e8bbd3de2c02bfcb0024daa778c1850bba9de94a2d1ec6a69"
Blend_url = "https://api.bland.ai/v1/calls"

# groq setup
GROQ_API_KEY = "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"
groq_url = "https://api.groq.com/openai/v1/chat/completions"

# Initialize Notion client
NOTION_TOKEN = "ntn_S6159294934albrajfceBHL4szrrrMllKAcFNUGM62v7JI"
DATABASE_ID = "18316f1f61d680a2921bd08b8c62f895"
notion = Client(auth=NOTION_TOKEN)

# call script setup
COLD_CALL_SCRIPT = "6a5a0412-6481-4533-b560-cf72283e956b"
CLOSER_CALL_SCRIPT = "29e7ef67-4d36-4d15-aa09-0a38642fea26"


# call general information
System_phone_number = "null"
Interruption_Threshold_in_ms = 100
LLM_temperature = 1
Agent_name = "Christin"
Cold_Agent_voice = "Public - June 2978"
Specialist_name = "David"
Closer_Agent_voice = "mason (da9f34)"

# ZOHO setup
ZOHO_CLIENT_ID = "1000.VO682Z1FM15RTS0EPC3QG9OR3ZA81J" # info@trueclaim.org client ID
ZOHO_CLIENT_SECRET = "73966cb9ddadac75baa08767dc1e8fd4caaa758b54" # info@trueclaim.org client secret
ZOHO_REFRESH_TOKEN = "1000.8e189778917bde78b6b8fd1fced5a6f8.b34bcab0a7248cf421fc8608c134348e"  # This is the long-lived token you received initially
ZOHO_ACCOUNT_ID = "3454657000000008002" # info@trueclaim.org Account ID
Zoho_auth_token = "1"
ZOHO_URL = f"https://mail.zoho.com/api/accounts/{ZOHO_ACCOUNT_ID}/messages"


function definition

In [66]:
def pull_notion_data(type="all"):
    """
    Retrieves and filters data from a Notion database based on the specified type.
    
    Args:
        type (str): The type of data to pull - "cold" for new/unanswered leads, 
                   "closer" for follow-up calls, or any other value for all data.
    
    Returns:
        pandas.DataFrame: A filtered dataframe containing the requested data.
    """
    
    # Initialize empty array to store the data
    data = []

    # Query the Notion database using the API
    response = notion.databases.query(database_id=DATABASE_ID)

    # Iterate through each page in the database and extract relevant information
    for page in response['results']:
        properties = page.get('properties', {})
        # Extract specific fields from the page properties
        row = {
            'notion raw id': page['id'],
            'customer name': page['properties']['First name']['title'][0]['plain_text'],
            'customer last name': page['properties']['Last name']['rich_text'][0]['plain_text'],
            'Phone': page['properties']['Main contact phone']['phone_number'],
            'Email': page['properties']['Main contact Email']['email'],
            'home address': page['properties']['Property Street']['rich_text'][0]['plain_text'],
            'Next Follow Up Date': page['properties']['Next Follow Up Date']['date']['start'] if page['properties']['Next Follow Up Date']['date'] else '',
            'Claim': page['properties']['Status of Claim']['status']['name'],
            'Lead': page['properties']['Status of lead']['status']['name'],   
            'call id' : page['properties']['Last call ID']['rich_text'][0]['plain_text'] if page['properties']['Last call ID']['rich_text'] else '',
            'cold call summery': page['properties']['cold call summery']['rich_text'][0]['plain_text'] if page['properties']['cold call summery']['rich_text'] else '',
            'Active': page['properties']['Active']['select']['name']
        }
        data.append(row)
        
    # Convert the collected data into a pandas DataFrame 
    df = pd.DataFrame(data)
        
    if type == "cold":
        # Convert date strings to datetime objects
        df['Next Follow Up Date'] = pd.to_datetime(df['Next Follow Up Date'])

        # Get current UTC time for comparison
        current_time = pd.Timestamp.now(tz='UTC')
        
        # Filter for new leads or leads that didn't answer and are past their follow-up date
        return df[((df['Lead'] == 'New') & (df['Active'] == 'Yes')) |
                  ((df['Lead'] == 'Did not answered') &
                   (df['Next Follow Up Date'] < current_time) &                 
                  (df['Active'] == 'Yes'))]                 
        
    elif type == "closer":
        pass
        
    else:    
        return df
        
def get_follow_up_date_groq(transcription, call_date):
    """
    Uses the Groq API to analyze a call transcript and suggest a follow-up date.
    
    Args:
        transcription (str): The transcript of the call to analyze
        call_date (str): The date when the call was made
    
    Returns:
        str: A suggested follow-up date in ISO 8601 format (YYYY-MM-DD HH:MM:SS)
    """
    # Set up the API headers with authentication
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    
    # Prepare the API request payload
    payload = {
        "model": "deepseek-r1-distill-llama-70b",
        "messages": [
            {
                "role": "system", "content": "You are an AI assistant tasked with analyzing call transcripts and suggesting specific follow-up dates, please resonse with ISO 8601 time and date format, for example: \"2025-02-06 12:00:00\", do not add any other text"
            },
            {
                "role": "user", "content": f"Based on this call transcript, suggest a specific follow-up date: {transcription}, the time of the call is{call_date}"
            }
        ]
    }
    
    # Make the API call and extract the suggested date
    groq_response = requests.post(groq_url, json=payload, headers=headers)
    return groq_response.json()['choices'][0]['message']['content'].strip().split('\n')[-1]
    
def update_data_base_after_cold_call(notion_raw_id, call_id):
    """
    Updates the Notion database with information from a completed cold call.
    
    Args:
        notion_raw_id (str): The Notion page ID to update
        call_id (str): The ID of the completed call
    """
    # Retrieve call details from the API
    call_id_response = json.loads(requests.request("GET", Blend_url+"/"+call_id, headers=headers).text)

    # Initialize dictionary to store call information
    call_info_dict = {}
    
    # Store basic call information
    call_info_dict["Last call date"] = call_id_response["created_at"]
    call_info_dict["Last call ID"] = call_id
    
    # Handle case where customer showed interest
    if "Interested" in call_id_response["pathway_tags"]:
        # Collect additional information for interested customers
        
        call_info_dict["Main contact Email collected"] = call_id_response["variables"]["email"]
        call_info_dict["Next Follow Up Date"]  = get_follow_up_date_groq(
            call_id_response['concatenated_transcript'],
            call_id_response["created_at"]
        )
        call_info_dict["Status of lead"] = call_id_response["pathway_tags"][0]
        call_info_dict["cold call summery"] = call_id_response["summary"].split("Here is a concise and insightful summary of the call:\n\n")[1]
        # Update Notion database with all collected information
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'cold call summery': {
                    'rich_text': [{'text': {'content': call_info_dict["cold call summery"]}}]
                },
                'Status of lead': {
                    'status': {'name': call_info_dict["Status of lead"]}
                },
                'Main contact Email collected': {
                    'email': call_info_dict["Main contact Email collected"]
                },
                'Next Follow Up Date': {
                    'date': {'start': call_info_dict["Next Follow Up Date"]}
                },
                'Last call ID': {
                    'rich_text': [{'text': {'content':call_info_dict["Last call ID"]}}]
                }
            }
        )
        
    # Handle case where customer did not answer
    elif not call_id_response["pathway_tags"]:
        # Set follow-up date to 2 days after the call

        # update Next Follow Up Date to 7 days after the call
        call_info_dict["Next Follow Up Date"] = (pd.to_datetime(call_id_response["created_at"]) + pd.Timedelta(days=2)).strftime('%Y-%m-%d %H:%M:%S')
        
        call_info_dict["Status of lead"] = 'Did not answered'
        
        # update notion table with new follow up time and lead status
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                },
                'Next Follow Up Date': {
                    'date': {
                        'start': call_info_dict["Next Follow Up Date"]
                    }
                }
            }
        )
        
    elif "Not interested" in call_id_response["pathway_tags"]:
        #collect trascription and update notion database
        
        # collect cold call transcription
        call_info_dict["cold call transcription"] = call_id_response["concatenated_transcript"] 
          
        call_info_dict["Status of lead"] = 'Not interested'
        
        # update notion table with new follow up time and lead status
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'cold call transcription': {
                    'rich_text': [
                        {
                            'text': {
                                'content': call_info_dict["cold call transcription"]
                            }
                        }
                    ]
                },
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                }
            }
        )
        
    elif "Asked for follow up" in call_id_response["pathway_tags"]:
        #collect follow up date and update notion DB
        
        # collect cold call transcription   
        call_info_dict["Next Follow Up Date"]  = get_follow_up_date_groq(call_id_response['concatenated_transcript'], call_id_response["created_at"])
        
        call_info_dict["Status of lead"] = 'Asked for follow up'
        
        # update notion table with new follow up time and lead status        
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                },
                'Next Follow Up Date': {
                    'date': {
                        'start': call_info_dict["Next Follow Up Date"]
                    }
                }
            }
        )
        
    else:
        # call got hang out for some reason, need to think what to do
        pass
            
def send_communication(notion_raw_id): 
    
    #collect all information from notion DB
    user_info_df = pull_notion_data("all")
  
    #filter specific raw id which just closed the call
    user_info_df = user_info_df[user_info_df['notion raw id'] == notion_raw_id]

    #send process intormation if lead is interested
    if user_info_df['Lead'].values[0] == 'Interested':
        global Zoho_auth_token

        print(f'sending post cold call process massage')
        # format the mail
        SUBJECT = f"True Claim Services claim process information for {user_info_df['customer name'].values[0]} {user_info_df['customer last name'].values[0]}"
    
        CONTENT = f"""
        <html>
            <body>
                <h1>Hello!</h1>
                <p>Hay there handsome :).</p>   
                <p>we will meet at {user_info_df['Next Follow Up Date'].values[0]}</p>    
    
            </body>
        </html>
        """
    
        # Headers
        headers = {
            "Authorization": f"Bearer {Zoho_auth_token}",
            "Content-Type": "application/json"
        }
        
        # Request body
        payload = {
            "fromAddress": "info@trueclaim.org",  # Your Zoho email
            "toAddress": user_info_df['Email'].values[0],
            "subject": SUBJECT,
            "content": CONTENT,
            "mailFormat": "html"
        }
    
    
        # try to send the mail
        response = requests.post(ZOHO_URL, headers=headers, json=payload)
        response = response.json()
    
        if response['status']['code'] == 200:
            print("Email sent successfully")
    
        # check if response failed, if so refresh token
        elif Zoho_auth_token == '1' or response['data']['errorCode'] == 'INVALID_OAUTHTOKEN':

            print("Invalide key, sendinng to refresh")
            
            # get new auth token
            
            Zoho_auth_token = refresh_access_token(ZOHO_REFRESH_TOKEN, ZOHO_CLIENT_ID, ZOHO_CLIENT_SECRET)
            # update header with new auth token
            headers = {
                "Authorization": f"Bearer {Zoho_auth_token}",
                "Content-Type": "application/json"
            }   
            
            #try to send again
            response = requests.post(ZOHO_URL, headers=headers, json=payload)
            
            # Check if request was successful
            if response.status_code == 200:
                print("Email sent successfully after token refreshed")
            else:
                print(f"Failed to send email. Status code: {response.status_code}")
                print(f"Response: {response.text}")

    elif user_info_df['Lead'].values[0] == 'Not interested':
        #send not interested campeign                   
        pass
        
    elif user_info_df['Lead'].values[0] == 'Did not answered':
        #send Did not answered Mail                   
        pass

def refresh_access_token(refresh_token, client_id, client_secret):
    """
    Get a new access token using the refresh token
    """
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    
    data = {
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "refresh_token"
    }
    
    response = requests.post(token_url, data=data).json()
    return response["access_token"]

def call_setup(row, type="cold"):     
               
    Customer_phone_number = row['Phone']
    Customer_name = row['customer name']
    Home_address = row['home address']
    notion_raw_id = row['notion raw id']  
    First_call_Summery  = row['cold call summery']    
        
                  
    headers = {
        'Authorization': BLEND_API_KEY,
        "Content-Type": "application/json"
    }
        
        
    if type == "cold":             

        data = {
            "phone_number": Customer_phone_number,
            "task": "",
            "model": "enhanced",
            "language": "en",
            "voice": Cold_Agent_voice,
            "voice_settings": {},
            "pathway_id": COLD_CALL_SCRIPT,
            "pathway_version": 1, # remove for production
            "local_dialing": False,
            "max_duration": "12",
            "answered_by_enabled": False,
            "wait_for_greeting": True,
            "noise_cancellation": True,
            "ignore_button_press": True,
            "record": False,
            "amd": False,
            "block_interruptions": True,
            "interruption_threshold": Interruption_Threshold_in_ms,
            "voicemail_message": "test",
            "temperature": LLM_temperature,
            "transfer_list": {},
            "pronunciation_guide": [],
            "request_data": {
            "customer name": Customer_name,
            "home address": Home_address,
            "agent name": Agent_name,
            },
            "retry": {
            "wait": 240,
            "voicemail_action": "hangup",
            "voicemail_message": ""
            },
            "dynamic_data": [],
            "analysis_schema": {},
            "calendly": {},
            "timezone": "America/New_York"
        }        
        print("Cold call setup is ready")       
            
    elif type == "closer":             

        data = {
            "phone_number": Customer_phone_number,
            "task": "",
            "model": "enhanced",
            "language": "en",
            "voice": Agent_voice,
            "voice_settings": {},
            "pathway_id": CALL_SCRIPT,
            "pathway_version": 1, # this is for testing, delete for production
            "local_dialing": False,
            "max_duration": "12",
            "answered_by_enabled": False,
            "wait_for_greeting": True,
            "noise_cancellation": True,
            "ignore_button_press": True,
            "record": False,
            "amd": False,
            "interruption_threshold": Interruption_Threshold_in_ms,
            "voicemail_message": "test",
            "temperature": LLM_temperature,
            "transfer_list": {},
            "pronunciation_guide": [],
            "request_data": {
            "customer name": Customer_name,
            "home address": Home_address,
            "agent name": Agent_name,
            "specialist name": Specialist_name,
            "first call transcription": First_call_Summery
            },
            "retry": {
            "wait": 720,
            "voicemail_action": "hangup",
            "voicemail_message": ""
            },
            "dynamic_data": [],
            "analysis_schema": {},
            "calendly": {},
            "timezone": "America/New_York"
        }          
        print("closer call setup is ready")               
   
    else:          
        data = {}        
        print("unkknown setup, please try again")    
                    
    return data, headers            
            

Main loop

In [ ]:

cold_call_df = pull_notion_data("cold")

# iterate rows and collect user data
for index, row in cold_call_df.iterrows():

    #call setup
    data, headers = call_setup(row, type="cold")
    # send call
    send_call_response = json.loads(requests.request("POST", Blend_url, json=data, headers=headers).text)

    # print status
    print(send_call_response)
    
    # collect call id
    call_id = send_call_response["call_id"]
    
    call_id_response = json.loads(requests.request("GET", Blend_url+"/"+call_id, headers=headers).text)
    # wait for call to end
    while call_id_response["completed"] == False:
        time.sleep(3)
        print("Call is still active")
        call_id_response = json.loads(requests.request("GET", Blend_url+"/"+call_id, headers=headers).text)
        
    # wait for tags to update
    time.sleep(3)
    
    # update database based on results
    update_data_base_after_cold_call(row['notion raw id'],call_id)

    # send correct mail
    send_communication(row['notion raw id'])
    
    
    
    

#Testing